In [ ]:
import streamlit as st
import torch
import io
import numpy as np
import clip
import torchvision.transforms as T
from PIL import Image
import cv2
import difflib
from torchvision import transforms
from torchvision.models import swin_t
import torch.nn as nn
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, AutoModelForSeq2SeqLM
from langdetect import detect
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas


**Load Swin Transformer Model**

In [ ]:
@st.cache_resource
def load_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = swin_t(weights=None)
    model.head = nn.Linear(model.head.in_features, 1)
    model.load_state_dict(torch.load("swin_lung_cancer.pth", map_location=device))
    model.to(device)
    model.eval()
    return model, device

model, device = load_model()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


**Load NLLB for Translation**

In [ ]:
@st.cache_resource
def load_nllb():
    model_name = "facebook/nllb-200-distilled-600M"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model

nllb_tokenizer, nllb_model = load_nllb()

def translate_nllb(text, source_lang, target_lang):
    inputs = nllb_tokenizer(text, return_tensors="pt")
    translated = nllb_model.generate(
        **inputs,
        forced_bos_token_id=nllb_tokenizer.convert_tokens_to_ids(target_lang),
        max_length=512
    )
    return nllb_tokenizer.decode(translated[0], skip_special_tokens=True)


**Load BioBERT for Q&A**

In [ ]:
@st.cache_resource
def load_biobert():
    tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
    model = AutoModelForQuestionAnswering.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
    return tokenizer, model

qa_tokenizer, qa_model = load_biobert()


**Load ViT Classifier**

In [ ]:
@st.cache_resource
def load_vit_classifier():
    from torchvision.models import vit_b_16
    vit = vit_b_16(pretrained=False)
    vit.heads.head = nn.Linear(vit.heads.head.in_features, 2)
    vit.load_state_dict(torch.load("vit_lung_classifier.pth", map_location="cpu"))
    vit.eval()
    return vit

vit_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

vit_labels = ["lung", "non-lung"]
vit_model = load_vit_classifier()


**Loading CLIP**

In [ ]:
@st.cache_resource
def load_clip_model():
    model, preprocess = clip.load("ViT-B/32", device="cpu")
    return model, preprocess

clip_model, clip_preprocess = load_clip_model()


**Language Options and UI Setup**

In [ ]:
nllb_languages = {
    "English": "eng_Latn",
    "Hindi (हिन्दी)": "hin_Deva",
    "French (Français)": "fra_Latn",
    "Spanish (Español)": "spa_Latn",
    "Tamil (தமிழ்)": "tam_Taml",
    "Bengali (বাংলা)": "ben_Beng",
    "Chinese (中文)": "zho_Hans",
    "Arabic (العربية)": "arb_Arab",
    "German (Deutsch)": "deu_Latn",
    "Japanese (日本語)": "jpn_Jpan",
    "Korean (한국어)": "kor_Hang",
    "Portuguese (Português)": "por_Latn",
    "Russian (Русский)": "rus_Cyrl",
    "Italian (Italiano)": "ita_Latn",
    "Dutch (Nederlands)": "nld_Latn",
    "Polish (Polski)": "pol_Latn",
    "Turkish (Türkçe)": "tur_Latn",
    "Vietnamese (Tiếng Việt)": "vie_Latn",
    "Thai (ไทย)": "tha_Thai",
    "Swahili (Kiswahili)": "swa_Latn",
    "Greek (Ελληνικά)": "ell_Grek",
    "Ukrainian (Українська)": "ukr_Cyrl",
    "Hebrew (עברית)": "heb_Hebr",
    "Romanian (Română)": "ron_Latn",
    "Czech (Čeština)": "ces_Latn",
    "Hungarian (Magyar)": "hun_Latn",
    "Swedish (Svenska)": "swe_Latn",
    "Finnish (Suomi)": "fin_Latn",
    "Norwegian (Norsk)": "nor_Latn",
    "Danish (Dansk)": "dan_Latn",
    "Malay (Bahasa Melayu)": "msa_Latn",
    "Indonesian (Bahasa Indonesia)": "ind_Latn",
    "Filipino (Filipino)": "fil_Latn",
    "Tamil - Sri Lanka (தமிழ் - இலங்கை)": "tam_Sinh",
    "Marathi (मराठी)": "mar_Deva",
    "Telugu (తెలుగు)": "tel_Telu",
    "Kannada (ಕನ್ನಡ)": "kan_Kann",
    "Gujarati (ગુજરાતી)": "guj_Gujr",
    "Punjabi (ਪੰਜਾਬੀ)": "pan_Guru",
    "Nepali (नेपाली)": "nep_Deva",
    "Sinhala (සිංහල)": "sin_Sinh",
    "Khmer (ភាសាខ្មែរ)": "khm_Khmr",
    "Laotian (ລາວ)": "lao_Laoo",
    "Mongolian (Монгол)": "mon_Cyrl",
    "Pashto (پښتو)": "pus_Afgh",
    "Farsi (Persian) (فارسی)": "fas_Pers",
    "Tigrinya (ትግርኛ)": "tir_Tigr",
    "Somali (Soomaali)": "som_Latn",
    "Amharic (አማርኛ)": "amh_Ethi",
    "Zulu (isiZulu)": "zul_Latn",
    "Xhosa (isiXhosa)": "xho_Latn",
    "Quechua (Runasimi)": "que_Latn",
    "Aymara (Aymar aru)": "aym_Latn",
    "Wolof (Wolof)": "wol_Latn",
    "Yoruba (Yorùbá)": "yor_Latn",
    "Igbo (Igbo)": "ibo_Latn",
    "Haitian Creole (Kreyòl Ayisyen)": "hat_Latn",
    "Basque (Euskara)": "eus_Latn",
    "Catalan (Català)": "cat_Latn",
    "Galician (Galego)": "glg_Latn",
    "Serbian (Српски)": "srp_Cyrl",
    "Croatian (Hrvatski)": "hrv_Latn",
    "Bosnian (Bosanski)": "bos_Latn",
    "Albanian (Shqip)": "sqi_Latn",
    "Bulgarian (Български)": "bul_Cyrl",
    "Macedonian (Македонски)": "mkd_Cyrl",
    "Slovak (Slovenčina)": "slk_Latn",
    "Slovenian (Slovenščina)": "slv_Latn",
    "Estonian (Eesti)": "est_Latn",
    "Latvian (Latviešu)": "lav_Latn",
    "Lithuanian (Lietuvių)": "lit_Latn",
    "Georgian (ქართული)": "geo_Geor",
    "Armenian (Հայերեն)": "hye_Latn",
    "Icelandic (Íslenska)": "isl_Latn",
    "Welsh (Cymraeg)": "cym_Latn",
    "Irish (Gaeilge)": "gle_Latn",
    "Scottish Gaelic (Gàidhlig)": "gla_Latn"
}

ui_texts = {
    "title": "🩻 Lung Cancer Detection & Multilingual Chatbot",
    "language_select": "🌐 Select your preferred language",
    "upload_ct_scan": "📤 Upload a CT scan image",
    "medical_report": "📝 Your Medical Report",
    "ask_question": "🤖 Ask a question about your report or general health",
    "submit_question": "💬 Your question",
    "export_pdf": "📥 Export Report as PDF",
    "telehealth_link": "💬 For further consultation, you can also talk to a doctor via telehealth: [Click here to consult](https://doctorondemand.com/)"
}

def translate_ui(text_dict, target_language):
    translated_texts = {}
    for key, text in text_dict.items():
        translated_texts[key] = translate_nllb(text, "eng_Latn", nllb_languages.get(target_language, "eng_Latn"))
    return translated_texts

**Rule-based FAQ Answering**

In [ ]:
def get_faq_answer(question):
    question = question.lower()
    if "symptoms" in question:
        return "Common symptoms of lung cancer include persistent cough, chest pain, shortness of breath, coughing up blood, and fatigue."
    elif "treatment" in question:
        return "Treatment options for lung cancer may include surgery, radiation therapy, chemotherapy, targeted therapy, or immunotherapy, depending on the type and stage of cancer."
    elif "diagnosed" in question:
        return "Lung cancer is diagnosed through imaging tests like CT scans, biopsies, and PET scans."
    elif "reduce risk" in question:
        return "To reduce your risk of lung cancer, avoid smoking, avoid secondhand smoke, and maintain a healthy diet and lifestyle."
    else:
        return "Sorry, I don't have an answer for that. Please consult a healthcare professional."


**Language Selection + UI Display**

In [ ]:
tgt_language = st.selectbox("🌐 Select your preferred language", list(nllb_languages.keys()))
tgt_lang_code = nllb_languages[tgt_language]
translated_ui = translate_ui(ui_texts, tgt_language)

st.title(translated_ui["title"])
st.markdown(f"**{translate_nllb('Upload your CT scan image:', 'eng_Latn', tgt_lang_code)}**")
st.markdown(translate_nllb("Drag and drop file here", "eng_Latn", tgt_lang_code))
st.markdown(translate_nllb("Limit 200MB per file • JPG, JPEG, PNG", "eng_Latn", tgt_lang_code))

**File Upload + CLIP + ViT Validation**

In [ ]:
uploaded_file = st.file_uploader(translated_ui["upload_ct_scan"], type=["jpg", "jpeg", "png"])

def is_non_medical_image(img, model, preprocess):
    classes_to_flag = ["dog", "cat", "person", "photo", "illustration", "drawing", "painting", "graph", "chart", "diagram", "scene", "landscape"]
    image_input = preprocess(img).unsqueeze(0)
    with torch.no_grad():
        text_inputs = clip.tokenize(classes_to_flag)
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarities = (100.0 * image_features @ text_features.T).squeeze(0)
        top_idx = torch.argmax(similarities).item()
    return classes_to_flag[top_idx]

def validate_ct_image(img, vit_model, vit_transform, labels, tgt_lang_code, translate):
    try:
        grayscale = img.convert("L")
        img_array = np.array(grayscale)
        height, width = img_array.shape
        if height < 150 or width < 150:
            return translate("This image is too small to be a CT scan. Please upload a larger image.", "eng_Latn", tgt_lang_code)
        edges = cv2.Canny(img_array, 50, 150)
        edge_density = np.sum(edges > 0) / edges.size
        if edge_density < 0.01 or edge_density > 0.2:
            return translate("This image has either too little or too much detail to be a CT scan.", "eng_Latn", tgt_lang_code)
        input_tensor = vit_transform(img.convert("RGB")).unsqueeze(0)
        with torch.no_grad():
            output = vit_model(input_tensor)
            pred_idx = torch.argmax(output, dim=1).item()
            pred_label = labels[pred_idx]
            if pred_label.lower() != "lung":
                return translate(f"This appears to be a scan of {pred_label}, not the lungs. Please upload a lung CT scan.", "eng_Latn", tgt_lang_code)
        return "Valid lung CT scan"
    except Exception:
        return translate("There was a problem processing your scan. Please upload a standard CT scan image.", "eng_Latn", tgt_lang_code)

def final_ct_validation(img, vit_model, vit_transform, vit_labels, tgt_lang_code):
    return validate_ct_image(img, vit_model, vit_transform, vit_labels, tgt_lang_code, translate_nllb)

if uploaded_file:
    img = Image.open(uploaded_file).convert("RGB")

    # Step 1: CLIP Check
    clip_label = is_non_medical_image(img, clip_model, clip_preprocess)
    if clip_label in ["dog", "cat", "person", "illustration", "drawing", "photo", "graph", "chart", "landscape"]:
        st.error(translate_nllb(f"❌ This image looks like a {clip_label} and is not a valid medical CT scan.", "eng_Latn", tgt_lang_code))
        st.stop()

    # Step 2: ViT Check
    msg = final_ct_validation(img, vit_model, vit_transform, vit_labels, tgt_lang_code)
    if "Valid lung CT scan" not in msg:
        st.error("❌ " + msg)
        st.stop()

    # Step 3: Show Image + Confirmation
    st.image(img, caption=translate_nllb("✅ Valid lung CT scan", "eng_Latn", tgt_lang_code), use_container_width=True)
    st.success(translate_nllb("✅ Valid lung CT scan", "eng_Latn", tgt_lang_code))
    st.info(translate_nllb("✅ Report generated successfully!", "eng_Latn", tgt_lang_code))

    # Step 4: Swin Prediction
    try:
        input_tensor = transform(img).unsqueeze(0).to(device)
    except Exception as e:
        st.error("⚠️ Unable to process the image. Please ensure it's a valid CT scan in PNG/JPEG format.")
        st.stop()

    with torch.no_grad():
        prediction = model(input_tensor).squeeze().item()

    def classify(score):
        if score < 1:
            return "Low"
        elif 1 <= score < 3:
            return "Moderate"
        else:
            return "High"

    category = classify(prediction)


**Medical Report + Visual Score + Expanders
python
Copy
Edit**

In [ ]:
# Generate report
translated_report = f"""
🩺 **{translate_nllb('Your Medical Report', 'eng_Latn', tgt_lang_code)}**

✅ **{translate_nllb('What Did We See in the Scan?', 'eng_Latn', tgt_lang_code)}**
{translate_nllb('The scan reveals features that may be consistent with potentially malignant nodules. These include irregular shapes, larger size, or increased density in certain lung regions. While this is not a definitive diagnosis, it does warrant further clinical evaluation. A follow-up with your physician is strongly recommended.' if category == "High"
else 'The scan shows some features that could be associated with early or borderline lung nodules. These may include small opacities, mildly irregular borders, or subtle density changes.' if category == "Moderate"
else 'The scan does not reveal any significant abnormalities in the lung tissue. No suspicious nodules or irregular patterns were detected.', 'eng_Latn', tgt_lang_code)}

📊 **{translate_nllb('Malignancy Risk Score', 'eng_Latn', tgt_lang_code)}**
{translate_nllb('Your AI-predicted risk score is', 'eng_Latn', tgt_lang_code)} **{round(prediction, 2)}**.
{translate_nllb('This puts your risk in the', 'eng_Latn', tgt_lang_code)} **{translate_nllb(category, 'eng_Latn', tgt_lang_code)}** {translate_nllb('category.', 'eng_Latn', tgt_lang_code)}

📌 **{translate_nllb('What is Malignancy Score?', 'eng_Latn', tgt_lang_code)}**
{translate_nllb('This is a number predicted by our AI model based on your lung scan. It indicates the likelihood of cancer:', 'eng_Latn', tgt_lang_code)}
- **Low (0–1)**: {translate_nllb('Minimal signs of cancer', 'eng_Latn', tgt_lang_code)}
- **Moderate (1–3)**: {translate_nllb('Some warning signs', 'eng_Latn', tgt_lang_code)}
- **High (3+)**: {translate_nllb('Strong suspicion, further tests recommended', 'eng_Latn', tgt_lang_code)}
"""

st.markdown(translated_report)

# Visual Risk Indicator
if category == "High":
    st.markdown(f'<div style="background-color:red;color:white;padding:5px;">{translate_nllb("High Risk", "eng_Latn", tgt_lang_code)} (Score: {round(prediction, 2)})</div>', unsafe_allow_html=True)
elif category == "Moderate":
    st.markdown(f'<div style="background-color:yellow;color:black;padding:5px;">{translate_nllb("Moderate Risk", "eng_Latn", tgt_lang_code)} (Score: {round(prediction, 2)})</div>', unsafe_allow_html=True)
else:
    st.markdown(f'<div style="background-color:green;color:white;padding:5px;">{translate_nllb("Low Risk", "eng_Latn", tgt_lang_code)} (Score: {round(prediction, 2)})</div>', unsafe_allow_html=True)

# Expanders
with st.expander("🔍 " + translate_nllb("Learn More: How was this score calculated?", "eng_Latn", tgt_lang_code)):
    st.markdown(translate_nllb("The malignancy score is based on an AI model trained using a large dataset of lung CT scans. It considers patterns in the scan that are often associated with cancerous growths. The model then calculates a risk score based on these patterns.", "eng_Latn", tgt_lang_code))

with st.expander("⚠️ " + translate_nllb("Learn More: What are the limitations of this prediction?", "eng_Latn", tgt_lang_code)):
    st.markdown(translate_nllb("The malignancy score is an AI-generated estimate and is not a definitive diagnosis. It can sometimes produce false positives or negatives. Always consult with a medical professional for a comprehensive evaluation.", "eng_Latn", tgt_lang_code))


**Q&A Section**

In [ ]:
st.subheader(translate_nllb("🤖 Ask a question about your report or general health", "eng_Latn", tgt_lang_code))
question = st.text_input(translate_nllb("💬 Your question", "eng_Latn", tgt_lang_code))

if question:
    try:
        lang_detected = detect(question)
    except:
        lang_detected = tgt_lang_code

    # Translate question
    question_translated = translate_nllb(question, tgt_lang_code, "eng_Latn")

    # Rule-based answer first
    faq_answer = get_faq_answer(question_translated)
    if faq_answer:
        translated_answer = translate_nllb(faq_answer, "eng_Latn", tgt_lang_code)
        st.markdown(f"🩺 **{translate_nllb('Answer', 'eng_Latn', tgt_lang_code)}:** {translated_answer}")
    else:
        # Use BioBERT fallback
        def get_biobert_answer(question, context):
            inputs = qa_tokenizer(question, context, return_tensors="pt")
            outputs = qa_model(**inputs)
            start = torch.argmax(outputs.start_logits)
            end = torch.argmax(outputs.end_logits)
            answer = qa_tokenizer.convert_tokens_to_string(
                qa_tokenizer.convert_ids_to_tokens(inputs.input_ids[0][start:end + 1])
            )
            return answer

        answer = get_biobert_answer(question_translated, translated_report).strip()
        if not answer or "[UNK]" in answer or len(answer) < 3:
            answer = "Sorry, I couldn't find a clear answer. Please consult a doctor for guidance."

        translated_answer = translate_nllb(answer, "eng_Latn", tgt_lang_code)
        st.markdown(f"🩺 **{translate_nllb('Answer', 'eng_Latn', tgt_lang_code)}:** {translated_answer}")


**Telehealth Link**

In [ ]:
telehealth_link = "https://doctorondemand.com/"
st.markdown(f"💬 **{translate_nllb('For further consultation, you can also talk to a doctor via telehealth', 'eng_Latn', tgt_lang_code)}:** [**{translate_nllb('Click here to consult', 'eng_Latn', tgt_lang_code)}**]({telehealth_link})")

**Export PDF**

In [ ]:
def export_pdf(report_text):
    from reportlab.lib.pagesizes import letter
    from reportlab.pdfgen import canvas
    import textwrap
    buffer = io.BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    c.setFont("Helvetica", 11)
    x, y = 40, 750
    for paragraph in report_text.split("\n"):
        lines = textwrap.wrap(paragraph, width=100)
        for line in lines:
            if y < 50:
                c.showPage()
                y = 750
                c.setFont("Helvetica", 11)
            c.drawString(x, y, line)
            y -= 15
        y -= 10
    c.save()
    buffer.seek(0)
    return buffer

if st.button(translated_ui["export_pdf"]):
    pdf_buffer = export_pdf(translated_report)
    st.download_button("Download PDF", pdf_buffer, file_name="lung_cancer_report.pdf", mime="application/pdf")
